In [1]:
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from huggingface_hub import login

login(token="YOUR-KEY-HUGGING-FACE")

In [3]:
from datasets import load_dataset

dataset = load_dataset("thailevann/durationQA_track8_vlsp2025")

README.md:   0%|          | 0.00/607 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/246k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/221 [00:00<?, ? examples/s]

In [4]:
def yn_to_binary(labels):
    return [1 if x.lower() == "yes" else 0 for x in labels]


File txt mẫu: https://drive.google.com/drive/folders/1DHSoPf-1BbA2oFrsP50weqW4SQh098-H

In [5]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score

# Đọc predictions
test_data = dataset["test"]
preds = []
with open("results.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        preds.append(json.loads(line))
y_true = [yn_to_binary(labels) for labels in test_data["labels"]]
y_pred = [yn_to_binary(item["labels"]) for item in preds]

print("Ground truth samples:", len(y_true))
print("Prediction samples:", len(y_pred))

# Map prediction theo ( question, context)
pred_map = {}
for p in preds:
    key = (p["question"], p["context"])
    pred_map[key] = p["labels"]

y_true, y_pred = [], []
missing = 0

for i in range(len(dataset["test"])):
    context = dataset["test"]["context"][i]
    question = dataset["test"]["question"][i]
    true_labels = dataset["test"]["labels"][i]

    key = (question, context)
    if key not in pred_map:
        missing += 1
        continue

    y_true.append(yn_to_binary(true_labels))
    y_pred.append(yn_to_binary(pred_map[key]))


if missing > 0:
    print(f"⚠️ Cảnh báo: thiếu {missing} mẫu trong results.txt (không tính vào điểm).")

if len(y_true) == 0:
    print("❌ Không có mẫu nào khớp giữa results.txt và dataset['test'].")
else:
    # Flatten để tính micro precision/recall/F1
    y_true_flat = sum(y_true, [])
    y_pred_flat = sum(y_pred, [])

    precision = precision_score(y_true_flat, y_pred_flat, zero_division=0)
    recall = recall_score(y_true_flat, y_pred_flat, zero_division=0)
    f1 = f1_score(y_true_flat, y_pred_flat, zero_division=0)

    # Exact Match
    exact_match = sum([t == p for t, p in zip(y_true, y_pred)]) / len(y_true)

    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"F1: {f1*100:.2f}%")
    print(f"Exact Match: {exact_match*100:.2f}%")



Ground truth samples: 221
Prediction samples: 221
Precision: 85.94%
Recall: 86.33%
F1: 86.14%
Exact Match: 74.66%
